In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Legend13")
import torch 
import torch.nn as nn
!pip install torchinfo
from torchinfo import summary

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!jupyter nbconvert --to markdown /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNet.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNet.ipynb to markdown
[NbConvertApp] Writing 31927 bytes to /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNet.md


## **Issue of Interest : Degradation**

<br/>

- As the depth of network increases, a degradation problem where deeper network produces lower training accuracy compared to its shallower couterpart become more pronounced.

- Previously, main concern regarding to stacking more layers to network is gradient vanishing and exploding problem, which has been addressed with batch normalization and normalized initialization (e.g. kaiming initialization). 

- Even without those vanishing gradient problems, accuracy of neural networt starts to gets saturated as layer goes deeper and shows higher training error than the shallower networks, which represents adding layers can lead to under-fitting of the network. <br/>

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/12a8a8c5-5764-49f3-8c14-2d6cbed4bf0b" width="500px">

<br/>

- The result above in the Figure 1. where 56-layer model recorded lower training error than 20-layer doesn't make sense because there's always a solution where 56-layer do the same thing for 20 layers and added layers are just identity mappings.

- Occurence of the degradation problem even with the existence of simple solution indicates that current deeper networks might have difficulties in approximating identity mappings by deeply stacked multiple layers.  

- Hence, the paper introduces "deep residual learning" where network learns to fit only residual part (difference between desired underlying mapping and input), instead of directly optimizing the underlying mappings

<br/>


&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img width="350" alt="image" src="https://github.com/SuminizZ/Physics/assets/92680829/97af9cd3-8d0b-41f1-8aa0-59855cc86baa">

<br/>

- $\large H(x) \approx x \,= F(x) + x$ &emsp; $\large H(x)$ : desired underlying mappings, &emsp;$\large F(x)$ : residual mappings, &emsp;$\large x$ : input

- This approach seems quite reasonable in that as the depth of network increases, the optimal role of single layer tends to be limited to make very small and accurate perturbations from the previous input with a resonable preconditioning x. (This assumption is quite validated in Figure 7. that shows standard deviations (std) of layer responses of ResNet model is smaller than plain model in genenral)

<br/>

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img width="500" alt="image" src="https://github.com/SuminizZ/Physics/assets/92680829/c1cb89a7-9524-4c65-9b67-7cadd7958c0e">

<br/>

- Under this network architecture, optimal function for identity mapping is simply zero instead of optimizing completely new funciton that approximates the input.

## **Skip Connection & Projection Connection**

- To implement residual learning, the researcher added direct shortcut from input to output of residual mapping function  <br/>

&emsp;&emsp;&emsp;&emsp;&emsp;$\large y = F(x, \{W_i\}) + x$ 

- The function $F(x, \{W_i\})$ represents the residual mapping to be learned that consists of two or more layers 

- Operations required to perform the shorcut connection is no more than a simple element-wise addition, which introduces neither extra parameters nor computational complexity 

- In case where the dimensions of F(x) changes from x due to the computations during residual mapping (e.g. convolution with stride 2 or convolution with larger feature maps), **projection shortcut** can be used to match the dimension. (typically done by 1x1 convolution with stride or matched featured maps) <br/>

&emsp;&emsp;&emsp;&emsp;&emsp;$\large y = F(x, \{W_i\}) + W_sx$


## **ResNet Architecture**

1. Comparison with VGG-19

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/f84ce266-6c7c-43f6-a1a2-f51841304cee" width="1000px"> 

<br/>

- solind lines represent skip connection and dashed lines represent projection lines 

2. Bottleneck Building Block 

- to reduce the number of parameters, first implement 1x1 convolution with reduced feature maps and then perform 3x3 convolution followed by again 1x1 convolution with increased feature dimension. 
    - replace 2-layer block (3x3 -> 3x3) into 3-layer bottleneck block (1x1 -> 3x3 -> 1x1)

- By using bottleneck blocks, 152-layer (11.3 bilion FLOPs) ResNet still has lower complexity compared to VGG-16/19 (15.3 bilion FLOPs) 

<br/>

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img width="500" alt="image" src="https://github.com/SuminizZ/Physics/assets/92680829/d3bf40a1-4133-4ca2-98c9-450c25075f6d"> 

<br/>

3. Downsampling
- performs down-sampling with 1x1 convolution with stride 2 at the first conv layer in each block 
    - ResNet v1.5 uses 3x3 conv with stride 2 instead of 1x1 to mitigate the loss of spatial information 


## **Architectures of ResNet**

<img width="800" alt="image" src="https://github.com/SuminizZ/Physics/assets/92680829/29d15105-a1f5-4485-a3aa-04f779e6c287">

## **Implementation with PyTorch**

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, projection=False):
        super().__init__()
        
        self.residual = nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),
                                    nn.BatchNorm2d(out_channels),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(out_channels, out_channels*self.expansion, 3, padding=1, bias=False),
                                    nn.BatchNorm2d(out_channels*self.expansion))    # enters into relu activation 
        if projection:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels*self.expansion, 1, stride=stride),
                                          nn.BatchNorm2d(out_channels*self.expansion))
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.residual(x)
        shortcut = self.shortcut(x)

        return self.relu(residual + shortcut)


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1, projection=False):
        super().__init__()

        self.residual = nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(out_channels, out_channels, 3, stride=stride, padding=1, bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(out_channels, out_channels*self.expansion, 1, bias=False),
                                   nn.BatchNorm2d(out_channels*self.expansion))
        if projection:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels*self.expansion, 1, stride=stride),
                                          nn.BatchNorm2d(out_channels*self.expansion))
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.residual(x)
        shortcut = self.shortcut(x)

        return self.relu(residual + shortcut)


class ResNet(nn.Module):
    def __init__(self, in_channels, block, expansion, block_repeats, num_classes, zero_init_residual=True):
        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 64, 7, stride=2, padding=3),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.conv2_pool = nn.MaxPool2d(3, stride=2, padding=1)

        self.expansion = expansion
        out_channels, self.conv2_blocks = self.stack_blocks(block, 64, block_repeats[0], 1)
        out_channels, self.conv3_blocks = self.stack_blocks(block, out_channels, block_repeats[1], 2)
        out_channels, self.conv4_blocks = self.stack_blocks(block, out_channels, block_repeats[2], 2)
        out_channels, self.conv5_blocks = self.stack_blocks(block, out_channels, block_repeats[3], 2)

        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Linear(out_channels, 1000)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            # zero-initialize the last BN in each residual branch -> set the start of residual branch as zero (identity mapping)
            # improves model by 0.2~0.3%p (https://arxiv.org/abs/1706.02677)
            elif isinstance(m, block):    
                nn.init.constant_(m.residual[-1].weight, 0)


    def stack_blocks(self, block, in_channel, block_repeat, stride):
        stacked = []

        c, repeats = block_repeat
        for _ in range(repeats):
            if stride == 2 or in_channel != c*self.expansion:
                stacked += [block(in_channel, c, stride, True)]
                in_channel = c*self.expansion
                stride = 1
            else:
                stacked += [block(in_channel, c)]
                in_channel = c*self.expansion

        return c*self.expansion, nn.Sequential(*stacked)  

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_pool(x)
        x = self.conv2_blocks(x)
        x = self.conv3_blocks(x)
        x = self.conv4_blocks(x)
        x = self.conv5_blocks(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        out = self.classifier(x)

        return out

## **Model Summary**

In [ ]:
block_repeats = {'resnet34' : [(64,3), (128,4), (256,6), (512,3)],
                'resnet101' : [(64,3), (128,4), (256,23), (512,3)],
                'resnet152' : [(64,3), (128,8), (256,36), (512,3)]}

model = ResNet(3, Bottleneck, 4, block_repeats['resnet152'], 1000)
summary(model, input_size=(2, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Sequential: 1-1                        [2, 64, 112, 112]         --
│    └─Conv2d: 2-1                       [2, 64, 112, 112]         9,472
│    └─BatchNorm2d: 2-2                  [2, 64, 112, 112]         128
│    └─ReLU: 2-3                         [2, 64, 112, 112]         --
├─MaxPool2d: 1-2                         [2, 64, 56, 56]           --
├─Sequential: 1-3                        [2, 256, 56, 56]          --
│    └─Bottleneck: 2-4                   [2, 256, 56, 56]          --
│    │    └─Sequential: 3-1              [2, 256, 56, 56]          58,112
│    │    └─Sequential: 3-2              [2, 256, 56, 56]          17,152
│    │    └─ReLU: 3-3                    [2, 256, 56, 56]          --
│    └─Bottleneck: 2-5                   [2, 256, 56, 56]          --
│    │    └─Sequential: 3-4              [2, 256, 56, 56]          70,400

In [ ]:
x = torch.randn(2,3,224,224)
out = model(x)
print(out.shape)
out

torch.Size([2, 1000])


tensor([[ 1.7212,  0.7408, -0.7578,  ..., -0.3376,  0.8207, -0.8635],
        [ 1.6384,  0.6766, -0.7177,  ..., -0.4407,  0.7628, -0.8020]],
       grad_fn=<AddmmBackward0>)

## **Interpretation of ResNet as Ensembles of Multiple Networks with Varying Depth**

<br/>

- There's a research that proposes an interpretation of the architecture of residual networks as a collection of many paths of differing length (depth) (https://arxiv.org/abs/1605.06431)

- ResNet contains $\large O(2^{n})$ implicit paths connecting input and output between each block (n equals to the number of block used) and adding one block in networks doubles the number of paths.

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/e25ae1c8-9757-4e10-8649-a5c7b3437034" width="700px">

<br/>

- ResNet shows ensemble-like behavior and paths in the networks do not necessarily depend on each other, which is true in that deleting multiple blocks from 54-blocks residual networks doesn't result in severe drop of test classification error. The performance of VGG networks, on the other hand, is significanlty affected by deleting a single layer as every path is connected across the entire depth in the VGG architecture.

<br/>

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/6b7ebe78-5c1c-4619-8458-792b3b4194a6" width="700px"> 

<br/>

&emsp;&emsp;&emsp;**Figure 3.**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/71a12b67-65fc-4a58-8f84-b9d5e52a2194" width="800px"> 

<br/>

- Tendency of paths in the residual networks to behave in an ensemble-like manner can also be shown with the following experiment where randomly deleting serveral modules smoothly (not abruptly) decreases the model performance. This result indicates that the performance of networks smoothly correlates with the number of valid paths, which is also the case in the ensemble model. 

<br/>

&emsp;&emsp;&emsp;**Figure 5.**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/c457b21c-6153-475d-a20e-800fcbb82c35" width="700px">

<br/>


### **Importance of Shorter Paths**

- Define the length of a path by the number of modules it passes throughout the entire networks. 

- Experiment in the paper shows that shorter paths play more crucial role in training the networks compared to longer paths

- **Distribution of length of paths of 54-layer ResNet** (Figure 6.)

    - Bernoulli distribution with mean path length equals to np = 54*(1/2) = 27
    - more than 95% of paths go through 19 to 35 modules.

<br/>

&emsp;&emsp;&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/9f109ee3-3bad-4ae1-b22f-3334b57d6716" width="400px">

<br/>

- **Magnitude (Norm) of gradient matrix of path with length k**

    - To empirically investigate the effect of vanishing gradients on residual networks, starting from 54 blocks, researchers sample individual paths of a certain length (k) and measure the norm of the gradient that arrives at the input. To sample a path of length k, they first feed a batch forward through the whole network. During the backward pass, k residual blocks are randomly sampled. For those k blocks, propagation is only through residual modules and the rest n-k blocks propagate through the skip-connection. Then, measure the norm of the gradient matrix of the single k length path and plot the result with varying k from 1 to 54.

    - To find **total gradient magnitude**, multiply frequency of each path length to the expected magnitude of gradient.
    
    <br/>

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/90f8bed6-5bba-4f0c-abb1-b692d9fba246" width="700px">

<br/>

- The results show that the gradient magnitude of a path decays roughly exponentially with the number of residual modules it went through in the backward pass.

- To summarize, each path in residual networks shows ensemble-like behavior, making independent contribution to the model and suprisingly, shorter lengthed paths (fewer residual modules passed trough back-propagation) has greater importance in the overall performance of networks.
